<a href="https://colab.research.google.com/github/teruyo14/style-aligned-lora-integration/blob/main/SharedAttention_StyleAligned_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/google/style-aligned
!mv style-aligned/* .
!pip install -r requirements.txt

Cloning into 'style-aligned'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 110 (delta 25), reused 21 (delta 21), pack-reused 81 (from 1)
Receiving objects: 100% (110/110), 15.87 MiB | 6.40 MiB/s, done.
Resolving deltas: 100% (47/47), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.6 MB/s eta 0:00:00


In [ ]:
!pip install gradio controlnet_aux -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.4/282.4 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.0 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import torch
from diffusers import StableDiffusionXLPipeline, DDIMScheduler
from safetensors.torch import load_file
import sa_handler
import gradio as gr

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
def initialize_pipeline(alpha=0.3):
    # Load the base model
    scheduler = DDIMScheduler(
        beta_start=0.00085,
        beta_end=0.012,
        beta_schedule="scaled_linear",
        clip_sample=False,
        set_alpha_to_one=False
    )

    pipeline = StableDiffusionXLPipeline.from_pretrained(
        "path/to/your/base-model",
        torch_dtype=torch.float16,
        use_safetensors=True,
        scheduler=scheduler
    )

    pipeline.enable_model_cpu_offload()

    # Load text embeddings
    pipeline.load_textual_inversion(
        "path/to/your/textual-inversion.safetensors",
        token="EasyNegativeV2"
    )

    # Load LoRA weights
    lora_path = "path/to/your/lora-model.safetensors"
    pipeline.load_lora_weights(lora_path, weight_name=None)

    # Settings for performance improvement
    pipeline.enable_vae_slicing()
    pipeline.enable_vae_tiling()

    handler = sa_handler.Handler(pipeline)

    sa_args = sa_handler.StyleAlignedArgs(
        share_group_norm=True,
        share_layer_norm=True,
        share_attention=True,
        adain_queries=True,
        adain_keys=True,
        adain_values=False,
    )

    handler.register(sa_args)

    pipeline.to("cuda")
    return pipeline


In [ ]:
# Function to generate images using the pipeline with default parameters
def generate_images(user_prompt, alpha=0.3, seed=0, width=512, height=960):
    # Initialize the pipeline
    pipeline = initialize_pipeline(alpha)

    # Set the random seed for reproducibility
    generator = torch.Generator(device="cuda").manual_seed(int(seed))

    # Set the fixed negative prompt
    set_of_negative_prompts = ["EasyNegativeV2" for i in range(5)]

    # Include the trigger words in the prompt
    trigger_words = "katou megumi"
    prompt = f"{trigger_words}, {user_prompt}"
    sets_of_prompts = [
        f"{trigger_words}, red haired girl, {user_prompt}",
        f"{trigger_words}, green haired girl, {user_prompt}",
        f"{trigger_words}, blue haired girl, {user_prompt}",
        f"{trigger_words}, purple haired girl, {user_prompt}",
        f"{trigger_words}, black haired girl, {user_prompt}",
    ]
    # Generate the image
    images = pipeline(
        sets_of_prompts,
        negative_prompt=set_of_negative_prompts,
        width=int(width),
        height=int(height),
        generator=torch.Generator().manual_seed(40),
    ).images

    return images


In [ ]:
# Create the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Girl Generation Demo with LoRA and Style-Aligned")

    with gr.Row():
        prompt_input = gr.Textbox(
            label="Prompt",
            lines=3,
            placeholder="Enter your prompt here"
        )
    with gr.Row():
        alpha_input = gr.Slider(
            label="LoRA Alpha Value",
            minimum=0.0,
            maximum=1.0,
            step=0.05,
            value=0.3
        )
    with gr.Row():
        seed_input = gr.Number(label="Seed (optional)", value=0, precision=0)
        width_input = gr.Slider(label="Width", minimum=256, maximum=1024, step=64, value=512)
        height_input = gr.Slider(label="Height", minimum=256, maximum=1024, step=64, value=960)
    generate_button = gr.Button("Generate Images")

    # Update output component to gr.Gallery to display multiple images
    output_gallery = gr.Gallery(label="Generated Images", show_label=False, columns=2)

    # Update the function called on button click
    def on_generate(prompts, alpha, seed, width, height):
        images = generate_images(
            user_prompt=prompts,
            alpha=alpha,
            seed=seed,
            width=width,
            height=height
        )
        return images

    # Connect the button click event to the function
    generate_button.click(
        fn=on_generate,
        inputs=[
            prompt_input,
            alpha_input,
            seed_input,
            width_input,
            height_input
        ],
        outputs=output_gallery  # Use the updated output component
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1f1dbaddd34443aa89.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
